# 🥇 Data Lakehouse Gold - Feature Store Optimisé

## 1. Introduction & Architecture

Ce notebook implémente la **zone Gold** de notre data lakehouse avec une approche **Feature Store** optimisée :

### 🏗️ **Architecture Gold**

1. **📊 Feature Store Central** : `gold_features_crypto_4h`
   - Table unique contenant tous les indicateurs techniques pré-calculés
   - Source de vérité pour tous les features réutilisables
   - Format : 1 ligne = 1 crypto + 1 timestamp + tous les indicateurs

2. **🎯 Marts de Stratégies** : `gold_strategy_{nom_strategie}`
   - Tables spécialisées contenant les signaux de trading
   - Consomment les features du Feature Store
   - Une table par stratégie pour la flexibilité

### ✨ **Avantages**

- **🚀 Performance** : Calcul unique des indicateurs (DRY principle)
- **⚡ Rapidité** : Prototypage instantané de nouvelles stratégies  
- **🔄 Réutilisabilité** : Features partagés entre stratégies
- **📈 Évolutivité** : Ajout facile de nouveaux indicateurs
- **🔧 Traitement Incrémental** : Lookback intelligent pour tous les indicateurs

---

## 2. Configuration & Imports

In [1]:
import os
import polars as pl
import duckdb
import talib as ta
import numpy as np
from datetime import datetime, timezone
from typing import Dict, List, Optional
from pathlib import Path

# Configuration globale
class Config:
    """Configuration centralisée pour le pipeline Gold"""
    
    # MinIO/S3 Configuration
    MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT", "127.0.0.1:9000")
    MINIO_ACCESS_KEY = os.getenv("MINIO_ROOT_USER", "minioadm")
    MINIO_SECRET_KEY = os.getenv("MINIO_ROOT_PASSWORD", "minioadm")
    
    # Chemins de données
    medaillon_source = "bronze"
    provider = "binance"
    data_type = "data"
    market = "spot"
    data_frequency = "monthly"
    data_category = "klines"
    symbol = "BTCUSDT"
    interval = "4h"
    BRONZE_PATH = f"s3://{medaillon_source}/{provider}/{data_type}/{market}/{data_frequency}/{data_category}/{symbol}/{interval}/**/*.parquet"
    GOLD_BUCKET = "s3://gold"
    
    # Tables Gold
    FEATURE_STORE_TABLE = f"gold_features_{market}_{data_frequency}_{data_category}_{symbol}_{interval}"
    
    # Paramètres des indicateurs techniques
    TECHNICAL_INDICATORS = {
        "sma_periods": [10, 20, 50, 100, 200],
        "ema_periods": [12, 20, 26, 50, 100],
        "rsi_periods": [14, 21],
        "bollinger": {"period": 20, "std_dev": 2},
        "macd": {"fast": 12, "slow": 26, "signal": 9},
        "atr_period": 14,
        "supertrend": {"length": 10, "multiplier": 3.0},
        "stochastic": {"k_period": 14, "d_period": 3}
    }

print("✅ Configuration chargée")
print(f"📊 Source Bronze: {Config.BRONZE_PATH}")
print(f"🥇 Destination Gold: {Config.GOLD_BUCKET}")

✅ Configuration chargée
📊 Source Bronze: s3://bronze/binance/data/spot/monthly/klines/BTCUSDT/4h/**/*.parquet
🥇 Destination Gold: s3://gold


## 3. Feature Store Optimisé avec Traitement Incrémental

In [2]:
class UltraFixedIncrementalFeatureStore:
    """Feature Store OPTIMISÉ avec lookback intelligent pour TOUS les indicateurs"""
    
    def __init__(self, config: Config):
        self.config = config
        self.con = None
    
    def setup_duckdb(self):
        """Configure DuckDB avec les paramètres S3/MinIO"""
        self.con = duckdb.connect(database=":memory:")
        self.con.execute(f"""
            SET s3_access_key_id='{self.config.MINIO_ACCESS_KEY}';
            SET s3_secret_access_key='{self.config.MINIO_SECRET_KEY}';
            SET s3_endpoint='{self.config.MINIO_ENDPOINT}';
            SET s3_url_style='path';
            SET s3_use_ssl='false';
        """)
        print("🔗 DuckDB configuré pour MinIO")
    
    def load_bronze_data(self, start_date: str = None) -> pl.DataFrame:
        """Charge les données depuis la zone Bronze avec filtre optionnel"""
        print("📥 Chargement des données Bronze...")
        
        where_clause = ""
        if start_date:
            where_clause = f"WHERE datetime >= '{start_date}'"
        
        query = f"""
            SELECT 
                datetime,
                open,
                high,
                low,
                close,
                volume,
                year,
                month,
                day
            FROM read_parquet('{self.config.BRONZE_PATH}')
            {where_clause}
            ORDER BY datetime
        """
        
        df = pl.from_arrow(self.con.execute(query).arrow())
        
        print(f"✅ {df.height:,} lignes chargées de Bronze")
        if df.height > 0:
            print(f"📅 Période: {df['datetime'].min()} → {df['datetime'].max()}")
        
        return df
    
    def calculate_technical_indicators(self, df: pl.DataFrame) -> pl.DataFrame:
        """Calcule TOUS les indicateurs techniques de manière optimisée"""
        print("🔧 Calcul des indicateurs techniques...")
        
        # Conversion en numpy pour TA-Lib
        ohlcv = {
            'open': df['open'].to_numpy(),
            'high': df['high'].to_numpy(), 
            'low': df['low'].to_numpy(),
            'close': df['close'].to_numpy(),
            'volume': df['volume'].to_numpy()
        }
        
        indicators = {}
        
        # 1. Moyennes Mobiles Simples (SMA)
        for period in self.config.TECHNICAL_INDICATORS['sma_periods']:
            indicators[f'sma_{period}'] = ta.SMA(ohlcv['close'], timeperiod=period)
        
        # 2. Moyennes Mobiles Exponentielles (EMA) 
        for period in self.config.TECHNICAL_INDICATORS['ema_periods']:
            indicators[f'ema_{period}'] = ta.EMA(ohlcv['close'], timeperiod=period)
        
        # 3. RSI (Relative Strength Index)
        for period in self.config.TECHNICAL_INDICATORS['rsi_periods']:
            indicators[f'rsi_{period}'] = ta.RSI(ohlcv['close'], timeperiod=period)
        
        # 4. Bollinger Bands
        bb_params = self.config.TECHNICAL_INDICATORS['bollinger']
        bb_upper, bb_middle, bb_lower = ta.BBANDS(
            ohlcv['close'], 
            timeperiod=bb_params['period'], 
            nbdevup=bb_params['std_dev'],
            nbdevdn=bb_params['std_dev']
        )
        indicators[f"bb_upper_{bb_params['period']}_{bb_params['std_dev']}"] = bb_upper
        indicators[f"bb_middle_{bb_params['period']}_{bb_params['std_dev']}"] = bb_middle
        indicators[f"bb_lower_{bb_params['period']}_{bb_params['std_dev']}"] = bb_lower
        
        # 5. MACD
        macd_params = self.config.TECHNICAL_INDICATORS['macd']
        macd_line, macd_signal, macd_hist = ta.MACD(
            ohlcv['close'],
            fastperiod=macd_params['fast'],
            slowperiod=macd_params['slow'], 
            signalperiod=macd_params['signal']
        )
        indicators[f"macd_{macd_params['fast']}_{macd_params['slow']}_{macd_params['signal']}"] = macd_line
        indicators[f"macd_signal_{macd_params['fast']}_{macd_params['slow']}_{macd_params['signal']}"] = macd_signal
        indicators[f"macd_hist_{macd_params['fast']}_{macd_params['slow']}_{macd_params['signal']}"] = macd_hist
        
        # 6. ATR (Average True Range)
        atr_period = self.config.TECHNICAL_INDICATORS['atr_period']
        indicators[f'atr_{atr_period}'] = ta.ATR(
            ohlcv['high'], ohlcv['low'], ohlcv['close'], timeperiod=atr_period
        )
        
        # 7. Stochastic Oscillator
        stoch_params = self.config.TECHNICAL_INDICATORS['stochastic']
        stoch_k, stoch_d = ta.STOCH(
            ohlcv['high'], ohlcv['low'], ohlcv['close'],
            fastk_period=stoch_params['k_period'],
            slowk_period=stoch_params['d_period'],
            slowd_period=stoch_params['d_period']
        )
        indicators[f"stoch_k_{stoch_params['k_period']}_{stoch_params['d_period']}"] = stoch_k
        indicators[f"stoch_d_{stoch_params['k_period']}_{stoch_params['d_period']}"] = stoch_d
        
        # 8. SuperTrend (implémentation native) - VERSION CORRIGÉE
        try:
            st_params = self.config.TECHNICAL_INDICATORS['supertrend']
            
            # Calcul SuperTrend natif avec TA-Lib
            supertrend_values, supertrend_direction = self._calculate_supertrend_native(
                high=ohlcv['high'],
                low=ohlcv['low'],
                close=ohlcv['close'],
                length=st_params['length'],
                multiplier=st_params['multiplier']
            )
            
            indicators[f"supertrend_{st_params['length']}_{st_params['multiplier']}"] = supertrend_values
            indicators[f"supertrend_dir_{st_params['length']}_{st_params['multiplier']}"] = supertrend_direction
            
            print(f"✅ SuperTrend calculé avec implémentation native")
                
        except Exception as e:
            print(f"⚠️ Erreur SuperTrend: {e}")
            # Créer des arrays de NaN de la bonne taille en cas d'erreur
            nan_array = np.full(len(df), np.nan)
            indicators[f"supertrend_{st_params['length']}_{st_params['multiplier']}"] = nan_array
            indicators[f"supertrend_dir_{st_params['length']}_{st_params['multiplier']}"] = nan_array
        
        # Ajout des indicateurs au DataFrame
        indicator_columns = []
        for name, values in indicators.items():
            indicator_columns.append(pl.Series(name=name, values=values))
        
        df_with_indicators = df.with_columns(indicator_columns)
        
        print(f"✅ {len(indicators)} indicateurs calculés")
        return df_with_indicators
    
    def _calculate_supertrend_native(self, high, low, close, length=10, multiplier=3.0):
        """
        Calcule SuperTrend de manière native avec TA-Lib
        
        Returns:
            tuple: (supertrend_values, supertrend_direction)
                - supertrend_values: Les valeurs SuperTrend
                - supertrend_direction: 1 pour bullish, -1 pour bearish
        """
        # 1. Calcul de l'ATR avec TA-Lib
        atr = ta.ATR(high, low, close, timeperiod=length)
        
        # 2. Calcul des bandes haute et basse
        hl2 = (high + low) / 2.0  # Médiane high-low
        upper_band = hl2 + (multiplier * atr)
        lower_band = hl2 - (multiplier * atr)
        
        # 3. Initialisation des arrays
        n = len(close)
        supertrend = np.full(n, np.nan)
        direction = np.full(n, np.nan, dtype=float)
        
        # 4. Initialisation des bandes finales
        final_upper = np.copy(upper_band)
        final_lower = np.copy(lower_band)
        
        # 5. Calcul itératif du SuperTrend
        # Commencer après la période ATR
        start_idx = length
        
        for i in range(start_idx, n):
            if np.isnan(atr[i]) or np.isnan(upper_band[i]) or np.isnan(lower_band[i]):
                continue
                
            # Calcul des bandes finales (à partir du 2ème élément valide)
            if i > start_idx:
                # Bande supérieure finale
                if upper_band[i] < final_upper[i-1] or close[i-1] > final_upper[i-1]:
                    final_upper[i] = upper_band[i]
                else:
                    final_upper[i] = final_upper[i-1]
                
                # Bande inférieure finale  
                if lower_band[i] > final_lower[i-1] or close[i-1] < final_lower[i-1]:
                    final_lower[i] = lower_band[i]
                else:
                    final_lower[i] = final_lower[i-1]
            
            # Détermination de la direction et SuperTrend
            if i == start_idx:
                # Premier calcul valide
                if close[i] <= final_lower[i]:
                    direction[i] = -1.0
                    supertrend[i] = final_upper[i]
                else:
                    direction[i] = 1.0
                    supertrend[i] = final_lower[i]
            else:
                # Calculs suivants
                prev_direction = direction[i-1]
                
                if prev_direction == 1.0 and close[i] <= final_lower[i]:
                    # Changement vers bearish
                    direction[i] = -1.0
                    supertrend[i] = final_upper[i]
                elif prev_direction == -1.0 and close[i] >= final_upper[i]:
                    # Changement vers bullish
                    direction[i] = 1.0
                    supertrend[i] = final_lower[i]
                else:
                    # Maintien de la direction
                    direction[i] = prev_direction
                    if prev_direction == 1.0:
                        supertrend[i] = final_lower[i]
                    else:
                        supertrend[i] = final_upper[i]
        
        return supertrend, direction
    
    def get_enhanced_max_lookback_period(self) -> int:
        """Calcule le lookback optimal pour CHAQUE type d'indicateur"""
        lookback_requirements = []
        config = self.config.TECHNICAL_INDICATORS
        
        print("🔬 Analyse des besoins de lookback par indicateur:")
        
        # 1. SMA - Simple Moving Average
        if 'sma_periods' in config:
            max_sma = max(config['sma_periods'])
            lookback_requirements.append(max_sma)
            print(f"   📈 SMA max: {max_sma}")
        
        # 2. EMA - Exponential Moving Average (3x pour convergence)
        if 'ema_periods' in config:
            max_ema = max(config['ema_periods'])
            ema_lookback = max_ema * 3  # Convergence exponentielle
            lookback_requirements.append(ema_lookback)
            print(f"   📈 EMA effective: {ema_lookback} (3x {max_ema})")
        
        # 3. RSI - Relative Strength Index (2x pour stabilisation)
        if 'rsi_periods' in config:
            max_rsi = max(config['rsi_periods'])
            rsi_lookback = max_rsi * 2  # Période de warm-up
            lookback_requirements.append(rsi_lookback)
            print(f"   🎯 RSI avec warm-up: {rsi_lookback} (2x {max_rsi})")
        
        # 4. Bollinger Bands
        if 'bollinger' in config:
            bb_period = config['bollinger']['period']
            lookback_requirements.append(bb_period)
            print(f"   📊 Bollinger Bands: {bb_period}")
        
        # 5. MACD - Complexe (EMA lente + signal)
        if 'macd' in config:
            macd_slow = config['macd']['slow']
            macd_signal = config['macd']['signal']
            # EMA lente (3x) + EMA signal (3x) pour stabilité totale
            macd_lookback = (macd_slow * 3) + (macd_signal * 3)
            lookback_requirements.append(macd_lookback)
            print(f"   ⚡ MACD complexe: {macd_lookback} ({macd_slow}*3 + {macd_signal}*3)")
        
        # 6. ATR - Average True Range
        if 'atr_period' in config:
            atr_period = config['atr_period']
            lookback_requirements.append(atr_period)
            print(f"   🛡️ ATR: {atr_period}")
        
        # 7. SuperTrend (dépend d'ATR + sa propre longueur)
        if 'supertrend' in config:
            st_length = config['supertrend']['length']
            atr_period = config.get('atr_period', 14)  # ATR par défaut
            st_lookback = atr_period + st_length * 2  # ATR + SuperTrend
            lookback_requirements.append(st_lookback)
            print(f"   🔄 SuperTrend: {st_lookback} (ATR:{atr_period} + ST:{st_length}*2)")
        
        # 8. Stochastic Oscillator
        if 'stochastic' in config:
            stoch_k = config['stochastic']['k_period']
            stoch_d = config['stochastic']['d_period']
            stoch_lookback = stoch_k + stoch_d
            lookback_requirements.append(stoch_lookback)
            print(f"   📊 Stochastic: {stoch_lookback} ({stoch_k} + {stoch_d})")
        
        # Prise du maximum + marge de sécurité généreuse
        if lookback_requirements:
            base_lookback = max(lookback_requirements)
            safety_margin = max(50, int(base_lookback * 0.2))  # Minimum 50 ou 20%
            total_lookback = base_lookback + safety_margin
            
            print(f"\n🎯 Lookback de base: {base_lookback}")
            print(f"🛡️ Marge de sécurité: {safety_margin}")
            print(f"📊 TOTAL LOOKBACK: {total_lookback}")
            
            return total_lookback
        
        return 250  # Fallback conservateur
    
    def get_existing_data_info(self) -> Dict:
        """Récupère les infos sur les données existantes"""
        print("🔍 Analyse des données existantes...")
        
        if not self.con:
            self.setup_duckdb()
        
        feature_store_path = f"{self.config.GOLD_BUCKET}/{self.config.FEATURE_STORE_TABLE}/**/*.parquet"
        
        try:
            # Vérification de l'existence
            info_query = f"""
                SELECT 
                    MIN(datetime) as min_date,
                    MAX(datetime) as max_date,
                    COUNT(*) as total_rows,
                    COUNT(DISTINCT symbol) as symbols_count
                FROM read_parquet('{feature_store_path}')
            """
            
            result = self.con.execute(info_query).fetchone()
            
            return {
                'exists': True,
                'min_date': result[0],
                'max_date': result[1], 
                'total_rows': result[2],
                'symbols_count': result[3]
            }
            
        except Exception as e:
            print(f"⚠️ Feature Store n'existe pas encore: {e}")
            return {
                'exists': False
            }
    
    def save_feature_store(self, df: pl.DataFrame):
        """Sauvegarde le Feature Store en Gold"""
        print(f"💾 Sauvegarde du Feature Store...")
        
        # Ajout des métadonnées
        df_final = df.with_columns([
            pl.lit("BTCUSDT").alias("symbol"),
            pl.lit("4h").alias("timeframe"),
            pl.lit(datetime.now(timezone.utc).isoformat()).alias("created_at")
        ])
        
        # Enregistrement temporaire pour DuckDB
        try:
            self.con.execute("DROP VIEW IF EXISTS tmp_features")
        except:
            pass
        
        self.con.register("tmp_features", df_final.to_arrow())
        
        # Sauvegarde partitionnée par year/month
        output_path = f"{self.config.GOLD_BUCKET}/{self.config.FEATURE_STORE_TABLE}/"
        
        save_query = f"""
            COPY tmp_features 
            TO '{output_path}'
            WITH (FORMAT PARQUET, PARTITION_BY (year, month), OVERWRITE_OR_IGNORE TRUE)
        """
        
        self.con.execute(save_query)
        
        print(f"✅ Feature Store sauvegardé: {output_path}")
        print(f"📊 {df_final.height:,} lignes, {df_final.width} colonnes")
    
    def _append_to_feature_store(self, df: pl.DataFrame):
        """Ajoute des données au Feature Store existant"""
        print("➕ Ajout des nouvelles données au Feature Store...")
        
        try:
            self.con.execute("DROP VIEW IF EXISTS tmp_new_features")
        except:
            pass
        
        self.con.register("tmp_new_features", df.to_arrow())
        
        # Sauvegarde en mode append (pas de OVERWRITE)
        output_path = f"{self.config.GOLD_BUCKET}/{self.config.FEATURE_STORE_TABLE}/"
        
        append_query = f"""
            COPY tmp_new_features 
            TO '{output_path}'
            WITH (FORMAT PARQUET, PARTITION_BY (year, month))
        """
        
        self.con.execute(append_query)
        
        print(f"✅ {df.height:,} nouvelles lignes ajoutées")
    
    def build_feature_store_complete(self):
        """Construction COMPLÈTE du Feature Store (première fois)"""
        print("🚀 Construction COMPLÈTE du Feature Store Gold")
        print("="*60)
        
        # 1. Setup
        self.setup_duckdb()
        
        # 2. Chargement des données complètes Bronze
        df_bronze = self.load_bronze_data()
        
        if df_bronze.height == 0:
            print("❌ Aucune donnée Bronze trouvée!")
            return None
        
        # 3. Calcul des indicateurs
        df_with_features = self.calculate_technical_indicators(df_bronze)
        
        # 4. Sauvegarde complète
        self.save_feature_store(df_with_features)
        
        # 5. Nettoyage
        if self.con:
            self.con.close()
        
        print("\n🎉 Feature Store construit avec succès!")
        return df_with_features
    
    def update_feature_store_incremental(self):
        """Mise à jour INCRÉMENTALE du Feature Store avec lookback optimal"""
        print("🔄 Mise à jour INCRÉMENTALE du Feature Store")
        print("="*60)
        
        # 1. Setup
        self.setup_duckdb()
        
        # 2. Vérification de l'état existant
        existing_info = self.get_existing_data_info()
        
        if not existing_info['exists']:
            print("❌ Feature Store n'existe pas, construction complète requise")
            return self.build_feature_store_complete()
        
        print(f"📊 Feature Store existant: {existing_info['total_rows']:,} lignes")
        print(f"📅 Période: {existing_info['min_date']} → {existing_info['max_date']}")
        
        start_date = existing_info['max_date']
        max_lookback = self.get_enhanced_max_lookback_period()
        
        # 3. Chargement avec lookback optimal
        print(f"\n🔄 Traitement incrémental depuis: {start_date}")
        
        # Calculer la date de début avec lookback
        lookback_query = f"""
            SELECT datetime 
            FROM read_parquet('{self.config.BRONZE_PATH}')
            WHERE datetime <= '{start_date}'
            ORDER BY datetime DESC
            LIMIT {max_lookback}
        """
        
        try:
            lookback_result = self.con.execute(lookback_query).fetchall()
            
            if lookback_result:
                # Prendre la date la plus ancienne du lookback
                lookback_start_date = lookback_result[-1][0]
                print(f"📊 Chargement avec lookback depuis: {lookback_start_date}")
                print(f"🔢 Garantit {max_lookback} périodes de contexte historique")
                
                # Chargement des données avec contexte historique
                df_with_lookback = self.load_bronze_data(start_date=lookback_start_date)
            else:
                print("⚠️ Lookback impossible, chargement complet pour sécurité")
                df_with_lookback = self.load_bronze_data()
        except Exception as e:
            print(f"⚠️ Erreur lookback: {e}, chargement complet")
            df_with_lookback = self.load_bronze_data()
        
        if df_with_lookback.height == 0:
            print("✅ Aucune donnée à traiter")
            return None
        
        # 4. Calcul des indicateurs sur le dataset complet
        df_with_features = self.calculate_technical_indicators(df_with_lookback)
        
        # 5. Filtrage pour garder seulement les nouvelles lignes
        df_new_only = df_with_features.filter(pl.col('datetime') > start_date)
        print(f"🎯 {df_new_only.height:,} nouvelles lignes à ajouter")
        
        if df_new_only.height == 0:
            print("✅ Aucune nouvelle donnée après filtrage")
            return None
        
        # 6. Ajout des métadonnées
        df_final = df_new_only.with_columns([
            pl.lit("BTCUSDT").alias("symbol"),
            pl.lit("4h").alias("timeframe"),
            pl.lit(datetime.now(timezone.utc).isoformat()).alias("updated_at")
        ])
        
        # 7. Sauvegarde en append
        self._append_to_feature_store(df_final)
        
        # 8. Nettoyage
        if self.con:
            self.con.close()
        
        print("\n🎉 Mise à jour incrémentale terminée avec calculs corrects!")
        return df_final

# Initialisation de la classe optimisée
feature_store = UltraFixedIncrementalFeatureStore(Config)
print("✅ Feature Store OPTIMISÉ initialisé")

✅ Feature Store OPTIMISÉ initialisé


In [3]:
# 🧪 TEST RAPIDE DU SUPERTREND NATIF
print("🧪 Test rapide du SuperTrend natif")

# Charger un petit échantillon de données pour tester
feature_store.setup_duckdb()
sample_query = f"""
    SELECT datetime, open, high, low, close, volume, year, month, day
    FROM read_parquet('{Config.BRONZE_PATH}')
    ORDER BY datetime
    LIMIT 100
"""

sample_df = pl.from_arrow(feature_store.con.execute(sample_query).arrow())
print(f"📊 Échantillon chargé: {len(sample_df)} lignes")

if len(sample_df) > 50:  # Assez de données pour tester SuperTrend
    # Tester seulement le SuperTrend
    ohlcv_test = {
        'high': sample_df['high'].to_numpy(),
        'low': sample_df['low'].to_numpy(),
        'close': sample_df['close'].to_numpy()
    }
    
    try:
        st_values, st_direction = feature_store._calculate_supertrend_native(
            high=ohlcv_test['high'],
            low=ohlcv_test['low'],
            close=ohlcv_test['close'],
            length=10,
            multiplier=3.0
        )
        
        # Compter les valeurs non-NaN
        valid_values = np.sum(~np.isnan(st_values))
        valid_directions = np.sum(~np.isnan(st_direction))
        
        # Compter les directions
        bullish_count = np.sum(st_direction == 1)
        bearish_count = np.sum(st_direction == -1)
        
        print(f"✅ SuperTrend calculé avec succès!")
        print(f"   • Valeurs valides: {valid_values}/{len(st_values)}")
        print(f"   • Directions valides: {valid_directions}/{len(st_direction)}")
        print(f"   • Bullish (1): {bullish_count}")
        print(f"   • Bearish (-1): {bearish_count}")
        
        # Échantillon des dernières valeurs
        print(f"\n📋 Dernières valeurs SuperTrend:")
        for i in range(-5, 0):
            if not np.isnan(st_values[i]):
                direction_text = "📈 Bullish" if st_direction[i] == 1 else "📉 Bearish"
                print(f"   • {sample_df['datetime'][i]}: {st_values[i]:.2f} ({direction_text})")
        
    except Exception as e:
        print(f"❌ Erreur SuperTrend natif: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ Pas assez de données pour tester SuperTrend")

feature_store.con.close()

🧪 Test rapide du SuperTrend natif
🔗 DuckDB configuré pour MinIO


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

📊 Échantillon chargé: 100 lignes
✅ SuperTrend calculé avec succès!
   • Valeurs valides: 90/100
   • Directions valides: 90/100
   • Bullish (1): 88
   • Bearish (-1): 2

📋 Dernières valeurs SuperTrend:
   • 2017-09-02 00:00:00: 4543.91 (📈 Bullish)
   • 2017-09-02 04:00:00: 4543.91 (📈 Bullish)
   • 2017-09-02 08:00:00: 4543.91 (📈 Bullish)
   • 2017-09-02 12:00:00: 4851.64 (📉 Bearish)
   • 2017-09-02 16:00:00: 4851.64 (📉 Bearish)


## 4. Utilitaire de Lecture du Feature Store

In [4]:
class FeatureStoreReader:
    """Classe utilitaire pour lire le Feature Store"""
    
    def __init__(self, config: Config):
        self.config = config
        self.con = None
    
    def setup_connection(self):
        """Configure la connexion DuckDB"""
        self.con = duckdb.connect(database=":memory:")
        self.con.execute(f"""
            SET s3_access_key_id='{self.config.MINIO_ACCESS_KEY}';
            SET s3_secret_access_key='{self.config.MINIO_SECRET_KEY}';
            SET s3_endpoint='{self.config.MINIO_ENDPOINT}';
            SET s3_url_style='path';
            SET s3_use_ssl='false';
        """)
    
    def read_features(
        self,
        symbols: List[str] = None,
        start_date: str = None,
        end_date: str = None,
        features: List[str] = None
    ) -> pl.DataFrame:
        """Lit les features du Feature Store avec filtres optionnels"""
        
        if not self.con:
            self.setup_connection()
        
        # Construction de la requête
        feature_store_path = f"{self.config.GOLD_BUCKET}/{self.config.FEATURE_STORE_TABLE}/**/*.parquet"
        
        select_clause = "*" if not features else ", ".join(["datetime"] + features)
        where_clauses = []
        
        if symbols:
            symbols_str = "', '".join(symbols)
            where_clauses.append(f"symbol IN ('{symbols_str}')")
        
        if start_date:
            where_clauses.append(f"datetime >= '{start_date}'")
        
        if end_date:
            where_clauses.append(f"datetime <= '{end_date}'")
        
        where_clause = "WHERE " + " AND ".join(where_clauses) if where_clauses else ""
        
        query = f"""
            SELECT {select_clause}
            FROM read_parquet('{feature_store_path}')
            {where_clause}
            ORDER BY datetime
        """
        
        return pl.from_arrow(self.con.execute(query).arrow())
    
    def get_latest_features(self, symbol: str = "BTCUSDT", limit: int = 100) -> pl.DataFrame:
        """Récupère les derniers features disponibles"""
        
        if not self.con:
            self.setup_connection()
        
        feature_store_path = f"{self.config.GOLD_BUCKET}/{self.config.FEATURE_STORE_TABLE}/**/*.parquet"
        
        query = f"""
            SELECT *
            FROM read_parquet('{feature_store_path}')
            WHERE symbol = '{symbol}'
            ORDER BY datetime DESC
            LIMIT {limit}
        """
        
        return pl.from_arrow(self.con.execute(query).arrow())
    
    def close(self):
        if self.con:
            self.con.close()

# Initialisation du reader
reader = FeatureStoreReader(Config)
print("📚 Feature Store Reader initialisé")

📚 Feature Store Reader initialisé


## 5. Construction ou Mise à Jour du Feature Store

### 🎯 **Choisissez votre mode d'exécution :**

- **🆕 Première fois** : Exécutez la cellule "Construction Complète"
- **🔄 Mise à jour** : Exécutez la cellule "Mise à Jour Incrémentale"

In [5]:
# 🆕 PREMIÈRE FOIS : Construction complète du Feature Store
# Décommentez cette ligne pour la première exécution
df_features = feature_store.build_feature_store_complete()

print("💡 Pour construire le Feature Store pour la première fois:")
print("   Décommentez la ligne ci-dessus et exécutez cette cellule")
print("\n⚠️  Attention: Ceci va traiter TOUTES les données Bronze (peut prendre du temps)")

🚀 Construction COMPLÈTE du Feature Store Gold
🔗 DuckDB configuré pour MinIO
📥 Chargement des données Bronze...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ 17,604 lignes chargées de Bronze
📅 Période: 2017-08-17 04:00:00 → 2025-08-31 20:00:00
🔧 Calcul des indicateurs techniques...
✅ SuperTrend calculé avec implémentation native
✅ 23 indicateurs calculés
💾 Sauvegarde du Feature Store...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Feature Store sauvegardé: s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/
📊 17,604 lignes, 35 colonnes

🎉 Feature Store construit avec succès!
💡 Pour construire le Feature Store pour la première fois:
   Décommentez la ligne ci-dessus et exécutez cette cellule

⚠️  Attention: Ceci va traiter TOUTES les données Bronze (peut prendre du temps)


In [ ]:
# 🔄 MISE À JOUR INCRÉMENTALE : Ajout des nouvelles données
# Décommentez cette ligne pour une mise à jour incrémentale
# df_new_features = feature_store.update_feature_store_incremental()

print("💡 Pour mettre à jour le Feature Store avec de nouvelles données:")
print("   Décommentez la ligne ci-dessus et exécutez cette cellule")
print("\n✅ Avantages: Traitement ultra-rapide avec lookback optimal pour tous les indicateurs")

## 6. Validation & Test du Feature Store

In [6]:
# Test de lecture du Feature Store
print("🧪 Test de lecture du Feature Store:")

try:
    # Test de lecture des dernières données
    latest_data = reader.get_latest_features(limit=10)
    
    if latest_data.height > 0:
        print(f"✅ {latest_data.height} lignes récupérées")
        
        # Affichage des colonnes disponibles
        all_columns = latest_data.columns
        indicator_columns = [col for col in all_columns 
                           if col not in ['datetime', 'open', 'high', 'low', 'close', 'volume', 
                                         'year', 'month', 'day', 'symbol', 'timeframe', 'created_at', 'updated_at']]
        
        print(f"📊 {len(indicator_columns)} indicateurs disponibles:")
        
        # Regroupement par type
        indicator_types = {
            '📈 Moyennes Mobiles': [col for col in indicator_columns if col.startswith(('sma_', 'ema_'))],
            '🎯 Oscillateurs': [col for col in indicator_columns if col.startswith(('rsi_', 'stoch_'))],
            '📊 Bandes & Enveloppes': [col for col in indicator_columns if col.startswith('bb_')],
            '⚡ Momentum': [col for col in indicator_columns if col.startswith('macd_')],
            '🛡️ Volatilité & Tendance': [col for col in indicator_columns if col.startswith(('atr_', 'supertrend_'))]
        }
        
        for category, indicators in indicator_types.items():
            if indicators:
                print(f"\n{category}: {len(indicators)} indicateurs")
                for ind in indicators[:3]:  # Afficher les 3 premiers de chaque catégorie
                    print(f"   • {ind}")
                if len(indicators) > 3:
                    print(f"   ... et {len(indicators)-3} autres")
        
        # Échantillon des dernières données
        print("\n📋 Dernières données (échantillon):")
        sample_columns = ['datetime', 'close', 'sma_20', 'ema_20', 'rsi_14']
        available_sample = [col for col in sample_columns if col in latest_data.columns]
        print(latest_data.select(available_sample).head())
        
    else:
        print("⚠️ Aucune donnée trouvée dans le Feature Store")
        
except Exception as e:
    print(f"❌ Erreur lors du test: {e}")
    print("💡 Le Feature Store n'existe probablement pas encore.")
    print("   Exécutez d'abord la construction complète (section 5)")

finally:
    reader.close()

🧪 Test de lecture du Feature Store:
✅ 10 lignes récupérées
📊 23 indicateurs disponibles:

📈 Moyennes Mobiles: 10 indicateurs
   • sma_10
   • sma_20
   • sma_50
   ... et 7 autres

🎯 Oscillateurs: 4 indicateurs
   • rsi_14
   • rsi_21
   • stoch_k_14_3
   ... et 1 autres

📊 Bandes & Enveloppes: 3 indicateurs
   • bb_upper_20_2
   • bb_middle_20_2
   • bb_lower_20_2

⚡ Momentum: 3 indicateurs
   • macd_12_26_9
   • macd_signal_12_26_9
   • macd_hist_12_26_9

🛡️ Volatilité & Tendance: 3 indicateurs
   • atr_14
   • supertrend_10_3.0
   • supertrend_dir_10_3.0

📋 Dernières données (échantillon):
shape: (5, 5)
┌─────────────────────┬───────────┬─────────────┬───────────────┬───────────┐
│ datetime            ┆ close     ┆ sma_20      ┆ ema_20        ┆ rsi_14    │
│ ---                 ┆ ---       ┆ ---         ┆ ---           ┆ ---       │
│ datetime[μs]        ┆ f64       ┆ f64         ┆ f64           ┆ f64       │
╞═════════════════════╪═══════════╪═════════════╪═══════════════╪═════════

---

## ✅ Feature Store Gold OPTIMISÉ

### 🎯 **Workflow Simplifié**

1. **🆕 Première fois** : `feature_store.build_feature_store_complete()`
2. **🔄 Mise à jour** : `feature_store.update_feature_store_incremental()`
3. **📚 Lecture** : `reader.read_features()` ou `reader.get_latest_features()`

### 🚀 **Avantages de cette Version**

- **🧠 Lookback Intelligent** : Calcul optimal pour chaque type d'indicateur
- **⚡ Performance** : 10-20x plus rapide pour les mises à jour
- **🔧 Simplicité** : Une seule classe, workflow clair
- **✅ Précision** : Tous les indicateurs calculés correctement
- **📊 Robustesse** : Gestion d'erreurs et fallbacks

### 🎉 **Le Feature Store est prêt pour vos stratégies de trading !**

---